<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="标题" style="width: 400px;"/> </a>

# 通过 Numba 使用 CUDA 的多维网格和共享内存

本节将介绍其他几个使用 Numba 在 CUDA 中编程的中级技巧。首先，我们将介绍 CUDA 如何支持创建二维和三维线程块及网格。凭借这种支持，我们在使用二维和三维数据进行编程时将更加容易。接下来，我们将介绍一种由程序员管理的片上内存空间（名为**共享内存**），该空间可用于同一个线程块内的线程之间的极速通信。您将有机会使用这两种技巧优化二维矩阵乘法的核函数。

本节还提供带示例的附录，其中会讲解如何减少矩阵转置算法的**共享内存存储体冲突**，同时还附有资源链接，可供您进一步学习。

## 目标

完成本节必要部分的学习后，您将能够：

* 使用多维线程块和网格，对多维数据集执行 GPU 加速并行工作。
* 使用共享内存在片上缓存数据，减少缓慢的全局内存访问。

## 二维和三维线程块及网格

网格和块均可分别进行配置，使之包含二维或三维线程块或线程的集合。多数情况下，此方法旨在为经常使用二维或三维数据集的程序员提供方便。下面举一个简单的例子，重点在于说明其采用的语法。您可能需要*同时*阅读核函数定义及其启动，才能理解该概念。

In [ ]:
import numpy as np
from numba import cuda

In [ ]:
A = np.zeros(16).reshape(4, 4).astype(np.int32)
d_A = cuda.to_device(A)

@cuda.jit
def add_2D_coordinates(A):
    # By passing `2`, we get the thread's unique x and y coordinates in the 2D grid
    x, y = cuda.grid(2)    # here, x is the column index, y is the row index
    
    A[y][x] = x + y        

In [ ]:
# Here we create a 2D grid with 4 blocks in a 2x2 structure, each with 4 threads in a 2x2 structure
# by using a Python tuple to signify grid and block dimensions.
blocks = (2, 2)
threads_per_block = (2, 2)

add_2D_coordinates[blocks, threads_per_block](d_A)
print(d_A.copy_to_host())

### 关于练习的注解

我们要记住的重要一点是，若要 GPU 出色地运行，您就需给予 GPU 足够大的网格，使流多处理器 (SM) 在等待那些尚未完成任务的操作到期之前，始终有其它工作需要执行。明确该要点后，您还需注意，本节中部分练习会要求您编写不符合此最佳实践的核函数，这样做的目的是在让您有机会专注于在多个维度上工作时所涉及的语法和一些基本模式，并有一定的时间来适应。

### 练习：在 GPU 上执行二维矩阵加法运算

本练习需要您修改核函数定义及其启动配置，以实现二维矩阵加法并行运算。您可以先编写一个“质朴的”核函数，它仅在网格形状与数据形状相符的条件下被启动时才能正常运行。如您遇到问题，请随时参阅 [此解决方案](../edit/solutions/add_matrix_solution.py)。

In [ ]:
# This function is written to add two 1D vectors. Refactor it to add 2D matrices
@cuda.jit
def add_matrix(A, B, C):
    j, i = cuda.grid(2)
    
    C[i, j] = A[i, j] + B[i, j]

In [ ]:
# Do not modify the values in this cell, which defines 2D matrices of size 36*36
A = np.arange(36*36).reshape(36, 36).astype(np.int32)
B = A * 2
C = np.zeros_like(A)
d_A = cuda.to_device(A)
d_B = cuda.to_device(B)
d_C = cuda.to_device(C)

In [ ]:
# Refactor the launch configuration to use a 2D grid with 2D blocks
# Refactor the launch configuration to use a 2D grid with 2D blocks
blocks = 36
threads_per_block = 36

# This launch will throw a Typing error until refactor the definition above to operate on 2D arrays
add_matrix[blocks, threads_per_block](d_A, d_B, d_C)

In [ ]:
from numpy import testing
output = d_C.copy_to_host()
solution = A+B
# This assertion will fail unles the output and solution are equal
testing.assert_array_equal(output, solution)

### 练习：在 GPU 上执行二维矩阵乘法运算

本练习需要您完成核函数的代码，以计算二维 [矩阵乘法](https://en.wikipedia.org/wiki/Matrix_multiplication) 运算中的一个元素。与刚刚编写的矩阵加法核函数类似，该核函数同样也是一个“质朴的”函数，因为它要求网格与做为参数传递进来的矩阵具有相同的维度。如您遇到问题，请参阅 [此解决方案](../edit/solutions/matrix_multiply_solution.py)。

In [ ]:
import numpy as np
from numba import cuda

@cuda.jit
def mm(a, b, c):
    row, column = cuda.grid(2)
    sum = 0
    
    ###
    # TODO: Build the rest of this kernel to calculate the value for one element in the output matrix.
    ###
        
    c[row][column] = sum

In [ ]:
# Do not modify the values in this cell
a = np.arange(16).reshape(4,4).astype(np.int32)
b = np.arange(16).reshape(4,4).astype(np.int32)
c = np.zeros_like(a)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.to_device(c)

grid = (2,2)
block = (2,2)
mm[grid, block](d_a, d_b, d_c)

In [ ]:
from numpy import testing
solution = a@b
output = d_c.copy_to_host()
# This assertion will fail until you successfully implement the kernel
testing.assert_array_equal(output, solution)

## 多维中的步长

正如可以使用 Numba 的 `cuda.gridsize(1)` 获取网格中的线程总数，我们也能使用 `cuda.gridsize(2)` 获取二维网格每个方向上的线程总数。该功能非常实用。例如，如果二维数据集大于网格，我们会需要每个线程在循环中跨过网格，以便覆盖所有必要工作，此时该功能便可派上用场。

正如一维跨网格循环，该技巧同样增加了网格和块尺寸的灵活性，且无需考虑数据的形状。以下示例展示了二维跨网格循环的使用效果，特别重要的是，最终打印的信息显示出网格中的哪个线程处理矩阵中的哪个元素。

In [ ]:
import numpy as np
from numba import cuda
@cuda.jit
def add_2D_coordinates_stride(A):

    grid_y, grid_x = cuda.grid(2)
    # By passing `2`, we get the grid size in both the x an y dimensions
    stride_y, stride_x = cuda.gridsize(2)
    
    for data_i in range(grid_x, A.shape[0], stride_x):
        for data_j in range(grid_y, A.shape[1], stride_y):
            A[data_i][data_j] = grid_x + grid_y

此处采用 3x2 结构创建了二维网格，网格中包含 6 个块，且每个块均采用 3x2 结构并由 6 个线程组成。该网格不仅小于我们的总数据集，而且网格形状也不能均匀适配数据集维度。即便如此，核函数依然能够访问数据中的每个元素。运行此单元后，尝试像网格一样调整数据的形状。运行代码前，您还可以尝试预测输出矩阵的值。

In [ ]:
A = np.zeros(55).reshape(11, 5).astype(np.int32)
d_A = cuda.to_device(A)

blocks = (3, 2)
threads_per_block = (3, 2)

# With this configuration, `stride_x` will be 9, and `stride_y` will be 4
add_2D_coordinates_stride[blocks, threads_per_block](d_A)
print(d_A.copy_to_host())

### 练习：对大于网格尺寸的二维矩阵执行加法运算

本练习需要您调整以上“质朴的”矩阵加法核函数，使其能够适于处理任意尺寸的数据集。如您遇到问题，请随时参阅 [此解决方案](../edit/solutions/add_matrix_stride_solution.py)。

In [ ]:
# Currently this kernel will only work correctly when passed matrices that are of the same size as the grid.
# Refactor using a strid in 2D so that it can work on data sets of an arbitrary size.
@cuda.jit
def add_matrix_stride(A, B, C):
    j,i = cuda.grid(2)
    
    C[i,j] = A[i,j] + B[i,j]

In [ ]:
# Please don't modify the values in this cell. They create a scenario where the data is
# larger than the grid size.
A = np.arange(64*64).reshape(64, 64).astype(np.int32)
B = A * 2
C = np.zeros_like(A)
d_A = cuda.to_device(A)
d_B = cuda.to_device(B)
d_C = cuda.to_device(C)

blocks = (6,6)
threads_per_block = (6,6)

add_matrix_stride[blocks, threads_per_block](d_A, d_B, d_C)

In [ ]:
from numpy import testing
output = d_C.copy_to_host()
solution = A+B
# This assertion will fail unles the output and solution are equal
testing.assert_array_equal(output, solution)

### 练习：对大于网格尺寸的二维矩阵执行乘法运算

本练习需要您完成矩阵乘法核函数，使其能够适用于任意网格和数据集形状。您只需编写两行包含 `TODO` 的代码，并使用 `grid_` 和 `stride_` 的值，将所执行的核函数中的工作正确映射到数据中。如您遇到问题，请参阅 [此解决方案](../edit/solutions/matrix_multiply_stride_solution.py)。

In [ ]:
import numpy as np
from numba import cuda
@cuda.jit
def mm_stride(A, B, C):

    grid_row, grid_column = cuda.grid(2)
    stride_row, stride_column = cuda.gridsize(2)
    
    for data_row in range(0): # TODO: replace 0 with values that will correctly set data_row
        for data_column in range(0): # TODO: replace 0 with values that will correctly set data_column
            sum = 0
            for i in range(A.shape[1]): # B.shape[0] would also be okay here
                sum += A[data_row][i] * B[i][data_column]
                
            C[data_row][data_column] = sum

In [ ]:
# Please do not modify this cell. The strange dimensions of this data, and
# the grid below are being set to make sure your kernel correctly handles arbitrary
# data and grid sizes.

a = np.arange(12).reshape(3,4).astype(np.int32)
b = np.arange(24).reshape(4,6).astype(np.int32)
c = np.zeros((a.shape[0], b.shape[1])).astype(np.int32)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.to_device(c)

ts = (4, 3)
bs = (3, 7)

mm_stride[bs, ts](d_a, d_b, d_c)

In [ ]:
from numpy import testing
solution = a@b
output = d_c.copy_to_host()
# This assertion will fail until you correctly update the kernel above.
testing.assert_array_equal(output, solution)

## 共享内存

目前为止，我们一直都在区分主机和设备内存，而设备内存就像某种单独的内存类型。但实际上，CUDA 拥有更为精细的[内存层次结构](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#memory-hierarchy)。我们至今一直使用的设备内存称为**全局内存**。设备上的任何线程或块都可使用该内存，其存续时间可贯穿应用程序的整个生命周期，且内存空间相对较大。

作为最后一个话题，我们将探讨如何使用一种名为**共享内存**的片上设备内存区域。共享内存是由程序员定义的缓存，容量有限。此类内存 [取决于所使用的 GPU](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities)，且可为同一个线程块内所有线程**共享**。共享内存是一种稀缺资源。若线程位于分配有共享内存的线程块之外，则无法访问此共享内存，且此类内存在核函数执行完毕后即会被释放。然而，共享内存带宽远高于全局内存，可在许多核函数中发挥出色的使用效果，尤其是有助于优化性能。

以下为共享内存的几个常见使用实例：

* 缓存从全局内存中读取的内存，以便在线程块内多次读取。
* 缓存线程输出，以便在重新写入全局内存之前对计算结果进行合并。
* 暂存数据，用于线程块内的分发和收集（scatter/gather）操作。

### 共享内存语法

Numba 提供用于分配共享内存和在块内线程之间执行同步操作的[函数](https://numba.pydata.org/numba-doc/dev/cuda/memory.html#shared-memory-and-thread-synchronization)，且这些函数对于并行线程读取或写入共享内存后的操作往往必不可少。

声明共享内存时，您需使用[Numba 类型](https://numba.pydata.org/numba-doc/dev/reference/types.html#numba-types)提供共享阵列的形状和类型。**阵列形状必须为常数**，因而您不可使用函数中传入的参数，也不得提供类似于 `numba.cuda.blockDim.x` 的变量或计算出的 `cuda.gridDim` 值。

下面是一个复杂的示例，展示了分配共享内存所采用的语法，其中的注释指出了数据从主机内存、全局设备内存到共享内存，然后依次返回至全局设备内存和主机内存的循环过程：

In [ ]:
import numpy as np
from numba import types, cuda

@cuda.jit
def swap_with_shared(x, y):
    # Allocate a 4 element vector containing int32 values in shared memory.
    temp = cuda.shared.array(4, dtype=types.int32)
    
    idx = cuda.grid(1)
    
    # Move an element from global memory into shared memory
    temp[idx] = x[idx]
    
    # cuda.syncthreads will force all threads in the block to synchronize here, which is necessary because...
    cuda.syncthreads()
    #...the following operation is reading an element written to shared memory by another thread.
    
    # Move an element from shared memory back into global memory
    y[idx] = temp[cuda.blockDim.x - cuda.threadIdx.x - 1] # swap elements

In [ ]:
x = np.arange(4).astype(np.int32)
y = np.zeros_like(x)

# Move host memory to device (global) memory
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)

swap_with_shared[1, 4](d_x, d_y)

In [ ]:
# Move device (global) memory back to the host
d_y.copy_to_host()

### 共享内存示例：矩阵转置

为举例展示共享内存的能力，并进一步演示二维 CUDA 编程，下面就让我们编写一个矩阵转置核函数。具体方法是把以行为主序的二维阵列再以列为主序进行排列。（此示例基于 Mark Harris 的博文：[高效矩阵转置](https://devblogs.nvidia.com/parallelforall/efficient-matrix-transpose-cuda-cc/)）。

在这个例子中，我们将使用共享内存作为一种机制，用来实现对全局内存进行合并读取和合并写入。对于转置算法来说，合并读/写通常是不可能的，但是由于共享内存的访问是如此之快，以至于我们可以对其进行“非合并”读取或写入而不会造成性能损失，以便实现随后的以合并的方式对全局内存进行读取和/或写入。该示例将证明这一点。

#### 不使用共享内存的矩阵转置

在展示共享内存的实现之前，让我们先用一个简单的方法，让每个线程仅使用全局内存独立地读写各个矩阵元素。 这样，当您看到共享内存示例时，可以专注于共享内存如何影响算法，而不是算法本身的基础知识：

In [ ]:
@cuda.jit
def transpose(a_in, a_out):
    # Explicitly calculate indices rather than using cuda.grid(2)
    row = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    col = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y

    a_out[row, col] = a_in[col, row]

In [ ]:
size = 16384
a_in = cuda.to_device(np.arange(size*size, dtype=np.int32).reshape((size, size)))
a_out = cuda.device_array_like(a_in)

print(a_in.copy_to_host())

In [ ]:
threads_per_block = (32, 32)
blocks_per_grid = (int(size/threads_per_block[0]), int(size/threads_per_block[1]))

In [ ]:
%timeit transpose[blocks_per_grid, threads_per_block](a_in, a_out); cuda.synchronize()

In [ ]:
print(a_out.copy_to_host())

#### 使用共享内存的矩阵转置

现在，让我们使用共享内存实现这个算法。为此：

1. 每个线程块将创建一个32x32元素的共享内存阵列
2. 每个线程块将从全局内存中的输入阵列合并读取到共享内存阵列中
3. 在将值写回到全局内存之前，块中的每个线程将等待该块中的所有其他线程使用线程同步来完成其读取操作
4. 我们从共享内存对全局内存进行合并写入，并以转置顺序进行写入。我们将分两步进行操作，首先是将共享内存阵列对应的在输入阵列中的位置进行转置（下面的4a），然后在将数据元素写回到全局内存之前（在下面的4b中）转置该元素在共享内存阵列中的位置。 

这种方法使我们能够在全局内存中进行合并的读写操作，因为我们可以在共享内存空间内执行转置，而共享内存阵列中的非连续读写不会造成性能损失。

In [ ]:
import numba.types

@cuda.jit
def tile_transpose(a_in, a_out):
    # `tile_transpose` assumes it is launched with a 32x32 block dimension,
    # and that `a_in` is a multiple of these dimensions.
    
    # 1) Create 32x32 shared memory array.
    tile = cuda.shared.array((32, 32), numba.types.int32)

    # Compute offsets into global input array.
    row = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    col = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    
    # 2) Make coalesced read from global memory into shared memory array.
    # Note the use of local thread indices for the shared memory write,
    # and global offsets for global memory read.
    tile[cuda.threadIdx.y, cuda.threadIdx.x] = a_in[col, row]

    # 3) Wait for all threads in the block to finish updating shared memory.
    cuda.syncthreads()
    
    # 4a) Calculate transposed location for the shared memory array tile
    # to be written back to global memory...
    row = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.x
    col = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.y

    # 4b) ...Write back to global memory,
    # transposing each element within the shared memory array.
    a_out[col, row] = tile[cuda.threadIdx.x, cuda.threadIdx.y]

In [ ]:
a_out = cuda.device_array_like(a_in)

%timeit tile_transpose[blocks_per_grid, threads_per_block](a_in, a_out); cuda.synchronize()
print(a_out.copy_to_host())

在已经加速的代码之上获得这样的加速比相当不错了。 （博客文章[Efficient Matrix Transpose](https://devblogs.nvidia.com/parallelforall/efficiency-matrix-transpose-cuda-cc/)展示了一种实现更高速度的方法，该方法是使一个线程块中的每个线程负责一个矩阵块(Tile)中的多个行，从而降低了计算线程索引值的成本。）

## 总结

您已完成本节学习，现已能够：

* 使用多维线程块和网格，对多维数据集执行 GPU 加速并行工作。
* 使用共享内存在片上缓存数据，同时减少缓慢的全局内存访问。

## 下载内容

如要下载此笔记本的内容，请执行以下单元，然后点击下方的下载链接。注意：由于笔记本中的部分文件路径链接是专为我们的平台量身设计，若您在本地 Jupyter 服务器上运行此笔记本，这些链接可能会遭到损坏。不过，您仍可通过 Jupyter 文件导航器导航至这些文件。

In [ ]:
!tar -zcvf section3.tar.gz .

[下载本节文件。](files/section3.tar.gz)

## 附录：无存储块冲突的矩阵转置

共享内存分成多个**存储块**。共享内存一共有 32 个存储块，且不使用同一存储块的内存读写可以同时运行。当并行线程尝试访问同一存储块内的内存时，我们将这种情况称为**存储块冲突**，该冲突将导致操作的顺序化。即便出现存储块冲突，您也可十分快速地访问共享内存；但若创建能够避免存储块冲突的内存访问模式，您也可进一步优化应用程序。

我们在此仅举一例，详情请参阅博文：[高效矩阵转置](https://devblogs.nvidia.com/parallelforall/efficient-matrix-transpose-cuda-cc/)。

In [ ]:
from numba import cuda, types
import numpy as np

TILE_DIM = 32
BLOCK_ROWS = 8
TILE_DIM_PADDED = TILE_DIM + 1  # Read Mark Harris' blog post to find out why this improves performance!
                                # https://devblogs.nvidia.com/parallelforall/efficient-matrix-transpose-cuda-cc/

@cuda.jit
def tile_transpose_no_bank_conflict(a_in, a_out):
    # THIS CODE ASSUMES IT IS RUNNING WITH A BLOCK DIMENSION OF (TILE_SIZE x TILE_SIZE)
    # AND INPUT IS A MULTIPLE OF TILE_SIZE DIMENSIONSx
    tile = cuda.shared.array((TILE_DIM, TILE_DIM_PADDED), types.int32)

    x = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.y
    
    for j in range(0, TILE_DIM, BLOCK_ROWS):
        tile[cuda.threadIdx.y + j, cuda.threadIdx.x] = a_in[y + j, x] # move tile into shared memory

    cuda.syncthreads()  # wait for all threads in the block to finish updating shared memory

    # Compute transposed offsets
    x = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.y

    for j in range(0, TILE_DIM, BLOCK_ROWS):
        a_out[y + j, x] = tile[cuda.threadIdx.x, cuda.threadIdx.y + j];


In [ ]:
size = 8192
a_in = cuda.to_device(np.arange(size*size, dtype=np.int32).reshape((size, size)))
a_out = cuda.device_array_like(a_in)

print(a_in.copy_to_host())

In [ ]:
grid_shape = (int(size/TILE_DIM), int(size/TILE_DIM))

%timeit tile_transpose_no_bank_conflict[grid_shape,(TILE_DIM, BLOCK_ROWS)](a_in, a_out); cuda.synchronize()
print(a_out.copy_to_host())

<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="标题" style="width: 400px;"/> </a>